# Разработка простого LLM-приложения

В этом разделе показан пример разработки LLM-приложения, которое переводит текст с английского языка на другой язык.
Итоговое приложение включает всего один вызов LLM плюс некоторую работу с промптами.
Это довольно простое приложение, но оно показывает, что множество функций можно реализовать только с помощью промптов и вызова LLM.

## Основные понятия

В разделе рассмотрены следующие основные понятия:

- использование [языковых моделей](/docs/concepts/#chat-models);
- использование [шаблонов промптов](/docs/concepts/#prompt-templates) и [парсеров вывода](/docs/concepts/#output-parsers);
- [объединение в цепочку](/docs/concepts/#langchain-expression-language) шаблона промптов + LLM + парсера вывода с помощью GigaChain

- Развертывание вашего приложения с [LangServe](/docs/concepts/#langserve).
<!--
- Отладка и трассировка вашего приложения с помощью [LangSmith](/docs/concepts/#langsmith)
-->

## Подготовка к разработке

### Jupyter-блокноты

Это руководство, как и большинство других в документации, использует [Jupyter-блокноты](https://jupyter.org/). Они отлично подходят для изучения работы с LLM-системами, так как предоставляют интерактивную среду для работы с руководствами и позволяют работать с непредвиденными ситуациями: недоступностью API, нетипичным выводом и другими.

Подробнее об установке jupyter — в [официальной документации](https://jupyter.org/install).

### Установка

Для установки GigaChain выполните команду:

<!--
### LangSmith

Многие приложения, которые вы создаете с помощью LangChain, будут содержать несколько шагов с многократными вызовами LLM.
По мере усложнения этих приложений становится важно иметь возможность инспектировать, что именно происходит внутри вашей цепочки или агента.
Лучший способ сделать это — с помощью [LangSmith](https://smith.langchain.com).

После регистрации по ссылке выше, убедитесь, что вы установили переменные среды для начала ведения журнала трассировок:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

Или, если вы работаете в блокноте, вы можете установить их с помощью:

```python
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```
-->

In [ ]:
pip install gigachain-community

Подробнее об установке — в разделе [Установка](https://developers.sber.ru/docs/ru/gigachain/get-started/installation).

## Работа с языковыми моделями

В первую очередь для работы использования GigaChain нужно подключить языковую модель.

Хотя GigaChain поддерживает различные языковые модели, основным преимуществом библиотеки является возможность работы с моделями GigaChat.

In [22]:
# | output: false
# | echo: false

from langchain_community.chat_models.gigachat import GigaChat

model = GigaChat(
    credentials="авторизационные_данные",
    scope="GIGACHAT_API_PERS",
    model="GigaChat",
    verify_ssl_certs=False,
)

:::note

Объект GigaChat принимает параметры:

- `credentials` — авторизационные данные. Строка, полученная в результате кодирования в Base64 клиентского идентификатора (Client ID) и ключа (Client Secret) API. Вы можете использовать готовые данные из личного кабинета или самостоятельно закодировать идентификатор и ключ.  О том как получить атворизационные данные — в разделе [Быстрый старт](/ru/gigachat/individuals-quickstart).
- `scope` — необязательный параметр, в котором можно указать версию API, к которой нужно обратиться. Возможные значения:
  
  - `GIGACHAT_API_PERS` — версия API для физических лиц;
  - `GIGACHAT_API_CORP` — версия API для ИП и юрлиц.

  По умолчанию запросы передаются в версию для физических лиц.

- `model` — необязательный параметр, в котором можно явно задать [модель GigaChat](/ru/gigachat/models).
- `verify_ssl_certs` — необязательный параметр, с помощью которого можно отключить проверку [сертификатов НУЦ Минцифры](/ru/gigachat/certificates).

:::

Попробуйте обратиться к модели напрямую.

Объекты `ChatModel` — это экземпляры Runnable-интерфейса GigaChain.
Все экземпляры Runnable предоставляют стандартный интерфейс для взаимподействия.

Так, чтобы обратиться к модели достаточно вызвать метод `.invoke()` со списком сообщений.

In [23]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Переведи следующее сообщение с русского на английский"),
    HumanMessage(content="привет!"),
]

model.invoke(messages)

AIMessage(content='Hello!', response_metadata={'token_usage': Usage(prompt_tokens=22, completion_tokens=3, total_tokens=25), 'model_name': 'GigaChat:3.1.25.3', 'finish_reason': 'stop'}, id='run-61e07c28-4b05-469c-97be-472b0f4e990f-0')

<!--
Если мы включили LangSmith, мы можем увидеть, что этот запуск зарегистрирован в LangSmith, и просмотреть [трассировку LangSmith](https://smith.langchain.com/public/88baa0b2-7c1a-4d09-ba30-a47985dde2ea/r).
-->

## Парсеры вывода OutputParsers

Ответ модели возвращается в форме сообщения `AIMessage`.
Он содержит сгенерированный текст и дополнительную информацию, например, количество затраченных токенов.
Зачастую для работы достаточно сгенерированного текста.
Чтобы получить его отдельно вы можете использовать парсер вывода.

Импортируйте простой парсер вывода.

In [8]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

Парсер можно использовать отдельно.
Например, вы можете сохранить результат вызова модели и затем передать его в парсер.

In [9]:
result = model.invoke(messages)

In [10]:
parser.invoke(result)

'Hello!'

Но чаще вам будет нужно соединять в цепочку парсер вывода и модель.
В таких случаях парсер вызывается каждый раз при обращении к цепочке.
Итоговая цепочка будет принимать на вход тип данных модели (строку или список сообщений) и возвращать тип данных парсера вывода (строка).

Вы можете создать цепочку с помощью оператора `|`, который используется в GigaChain для собъединения двух и более компонетов.

In [11]:
chain = model | parser

In [12]:
chain.invoke(messages)

'Hello!'

<!--
Если мы теперь посмотрим на LangSmith, мы увидим, что цепочка состоит из двух шагов: сначала вызывается языковая модель, затем результат передается парсеру вывода. Мы можем увидеть [трассировку LangSmith](https://smith.langchain.com/public/f1bdf656-2739-42f7-ac7f-0f1dd712322f/r).
-->

## Шаблоны промптов

Сейчас список сообщений в примере передается напрямую в модель.
Сам список сообщений, как правило, формируется логикой приложения из ввода пользователя.
Обычно такая логика принимает необработанный ввод и преобразует его в список сообщений, готовых для передачи в языковую модель.
Типичные преобразования включают добавление системного сообщения или изменение шаблона с учетом ввода пользователя.

Шаблоны промптов (PromptTemplates) — это конструкции GigaChain, которые принимают необработанный ввод пользователя и возвращают данные (промпт), готовые для передачи в языковую модель.

Создайте шаблон промпта, который будет принимать две пользовательские переменные:

- `language` — язык, на который нужно перевести текст;
- `text` — текст для перевода.

Для этого импортируйте `ChatPromptTemplate`:

In [13]:
from langchain_core.prompts import ChatPromptTemplate

Создайте строку, которая будет оформлена как системное сообщение:

In [14]:
system_template = "Переведи следующий текст на {language}:"

Теперь вы можете создать шаблон промпта.
Он будет состоять из комбинации `system_template` и шаблона для ввода текста.

In [15]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

Входными данными для этого шаблона является словарь.
Вы можем поэкспериментировать с этим шаблоном, чтобы увидеть, что он делает сам по себе.

In [16]:
result = prompt_template.invoke({"language": "английский", "text": "привет"})

result

ChatPromptValue(messages=[SystemMessage(content='Переведи следующий текст на английский:'), HumanMessage(content='привет')])

Шаблон возвращает объект `ChatPromptValue`, который состоит из двух сообщений.
Доступ к сообщениям можно получить с помощью метода `.to_messages()`:

In [17]:
result.to_messages()

[SystemMessage(content='Переведи следующий текст на английский:'),
 HumanMessage(content='привет')]

## Соединение всех компонентов вместе

Теперь объедините все три компонента вместе: шаблон, модель и парсер вывода с помощью оператора `|`.

In [18]:
chain = prompt_template | model | parser

In [19]:
chain.invoke({"language": "английский", "text": "привет"})

'Hello'

Это простой пример использования [LangChain Expression Language (LCEL)](/docs/concepts/#langchain-expression-language-lcel) для соединения вместе модулей GigaChain.
В таком подходе есть ряд преимуществ. В частности, он обеспечивает оптимизированную работу с потоковой передачей.

<!--
Если мы посмотрим на трассировку LangSmith, мы увидим все три компонента в [трассировке LangSmith](https://smith.langchain.com/public/bc49bec0-6b13-4726-967f-dbd3448b786d/r).
-->

## Развертывание с GigaServe

GigaServe помогает развертывать цепочки GigaChain в виде REST API.
Использовать GigaServe для работы с GigaChain необязательно.
Тем не менее ниже приводится пример, как вы можете развернуть приложение с помощью GigaServe.

Пример использует Python-файл, работа с которым выполняется с помощью командной строки.

Установка GigaServe:

In [ ]:
pip install "gigaserve[all]"

### Сервер

Для создания сервера для приложения, создайте файл `serve.py`.
Файл будет содержать логику для развертывания приложения.
Файл состоит из трех частей:

1. Определение цепочки, которую вы создали выше.
2. Приложение FastAPI.
3. Определение пути для доступа к цепочки с помощью `langserve.add_routes`.

```python
#!/usr/bin/env python
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models.gigachat import GigaChat
from langserve import add_routes

# 1. Создание шаблона промпта
system_template = "Переведи следующий текст на {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

# 2. Создание модели
model = GigaChat(credentials="авторизационные_данные", scope="GIGACHAT_API_PERS", model="GigaChat-Pro", verify_ssl_certs=False)

# 3. Создание парсера
parser = StrOutputParser()

# 4. Создание цепочки
chain = prompt_template | model | parser

# 4. Определение приложения
app = FastAPI(
  title="GigaChain Server",
  version="1.0",
  description="Простой сервер API, использующий Runnable-интерфейсы GigaChain.",
)

# 5. Добавление пути цепочки

add_routes(
    app,
    chain,
    path="/chain",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)
```

Запустите файл:
```bash
python serve.py
```
Вы должны увидеть, что ваша цепочка доступна по адресу [http://localhost:8000](http://localhost:8000).

### Песочница

Приложения GigaServe предоставляют доступ к простому [пользовательскому интерфейсу](https://github.com/langchain-ai/langserve/blob/main/README.md#playground) для настройки и вызова приложения с потоковым выводом и отображением промежуточных шагов.
Вы можете попробовать его по адресу [http://localhost:8000/chain/playground/](http://localhost:8000/chain/playground/).
Задай такие же входные данные — `{"language": "английский", "text": "привет"}` — приложение должно ответить как и раньше.

### Клиент

Для настройки клиентской части используйте [`langserve.RemoteRunnable`](/docs/langserve/#client).
Так вы сможете взаимодействовать с доступной цепочкой так, как если бы она выполнялась на стороне клиента.

In [21]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "английский", "text": "привет"})

'Hello'

Подробнее — в [документации GigaServe](/docs/langserve).

## Смотрите также

Чтобы глубже погрузиться в разработку приложений с помощью GigaChain, ознакомьтесь с разделами:

- [Обучающие материалы](/docs/tutorials);
- [Руководства](/docs/how_to);
- [Основные понятия](/docs/concepts).